## Introduction

- We only consider 1v1 UNO

- 2 states representations: naive representation for tabular methods (where you count all possibilities) results in way too large a state space to run tabular methods (if you do not even account for values this results in ~130M possible states). Needs reducing the state space through simplification --> 4 planes where each entry of a plane corresponds to a given card and 3 of those planes to having a given number of a specific card (you can either have 0, 1, or 2 of each card) ; the last plane is the target / open card.

- Motivations of using function approximation: state space is way too big.

- In total, there are 61 different actions a player can take, which are playing each of the 60 different cards and drawing a card from the deck.

- The reward is defined as +1 for winning, −1 for losing, and 0 for all intermediate states. Also we assume the game is endless so all final rewards are either +1 or -1.

## Agents

- Random vs. Random: we observe that the advantage given by starting the game is not negligible (the random player that starts has an average reward of .02 if I remember). This is not surprising though: in a game where the goal is to let go of your cards as fast as possible, getting rid of cards in a first place will result in higher rewrad in the long run.

- For each agent: 
    - agent vs. random
    - winning rate
    - learning curve

Play against each other
- Table

## Discussion & Limitation

#### Hyperparameters & Architecture

A majority of the agents we trained do not seem to have any edge over the random agent, which sounds very frustrating considering UNO does not look that complicated after all. A lot of parameters come into play regarding the performance of the agents we propose here however, namely the architecture of our NNs and the hyperparameters.

Most of our fine-tuning is heavily inspired by this paper (Winning UNO with reinforcement learning). In that regard, we believe a two-layer connected neural network should be deep enough to report good results for either of the algorithms we used. It is worth noting that the hyperparameters they use give a lot of importance to exploration throughout the training, even with decay ($\epsilon =  0.95, \kappa = 0.995$ with decay every 10th of the way still results in a very explorative behavior: $\epsilon \times \kappa^{10} \approx 0.90$???). The discount rate is high ($\gamma = 0.95$) considering winning rapidly or in a long time does not matter that much in the end, and the learning rate is $\alpha=0.0001$.

[Not sure about all that] The limited successes of some of our agents might stem from this aversion of some of the algorithms to exploration. For instance, REINFORCE does not give way to exploration. As the state space is really large, the policy might focus too much on specific episodes and states which have repeated throughout the training???

#### Multiple players

We have pondered over adding another random agent to solidify the training of our agents. Although the drawbacks of such a solution seem obvious (training will take longer - why should it be any different from 1v1 in terms of policy), adding players might result in more variance in the states covered throughout the episodes as there is more interaction between all players: in the end, this could reflect a better exploration throughout the training eventually reporting better results

#### Two more base agents

Some basic strategies come to mind for other baseline agents: one, which is widely played, is to play a card of the same value whenever it is possible, whatever the color is("value"-strategy), and an other one could be to play the target color whenever possible ("color"-strategy). The value-strategy usually pans out better in the end because there is ...

We havent added the "saying UNO" part but the obvious modeling is adding a Bernoulli variable when left with one card only: as this would be symmetric for all players, it wouldn't change the average reward, just takes longer training cause of the variance added.

#### State representation

There are a lot of possible state representations of UNO -->


#### Reward
The reward is defined as +1 for winning, −1 for losing, and 0 for all intermediate states

#### Game is difficult

Despite these modifications, Uno is a very stochastic game, and agents struggle to win over 60% of their games against random agents. This is very frustrating considering UNO does not seem to be that difficult of a game to play.



#### Double-Q Network

The Double Deep Q-Network (Double DQN) is an extension of the standard Deep Q-Network (DQN) that aims to address the overestimation bias often found in Q-learning algorithms. In traditional DQN, the same network is used to both select and evaluate the best action, which can lead to an overoptimistic estimation of action values. To mitigate this issue, the Double DQN employs two separate networks: one for action selection and another for action evaluation. During each iteration, the agent plays n games using ε-greedy exploration and generates a set of trajectories (s, a, r, s'). The Double DQN update rule then utilizes both networks by selecting the action with the highest Q-value from the first network and evaluating that action using the second network. This decouples the action selection and evaluation processes, reducing the overestimation bias and improving the stability of learning.

dqn agent was trained with a set of different parameters:
Number of episodes: 100k, 80k, 50k,
decaying epsilon from 0.95 to 0.01 with decaying factor 0.95, updating every 1000 episodes.
Constant epsilon: 0.05,0.5,0.8.








